# Machine Learning Challenge

Below are 2 data challenges that test for your ability to:
- Wrangle/clean data to make it usable by a model
- Figure out how to set up X's and y's for a use case, given a dataset
- Write code to robustly and reproducibly preprocess data
- Pick/design the right model, and tune hyperparameters to get the best performance

You can use any programming language, model, and package to solve these problems. Let us know of any assumptions you make in your process.

#### Deliverables:
- A link to a github repository that contains:
    - Clearly commented code that was written to solve these problems
    - Your trained models stored in a file (`.pkl`, `.h5`, `.tar` - whatever is appropriate). The models must have `predict(X)` functions. 
    - A readme file that contains:
        - Instructions to easily access/load the above
        - A writeup explaining any significant design decisions and your reasons for making them. 
        - If needed, a brief writeup explaining anything you are particularly proud of in your implementation that you might want us to focus on

#### How we'll assess your work:
- Accuracy/RMSE of your model when predicting on held-out data
- How well various edge cases are handled when testing on held-out data. For example, if the held-out data contains:
    - A new column that wasn't present in the dataset given to you
    - New value in a categorical field that wasn't seen in the dataset given to you
    - NA values
- Efficiency of the code. 
    - Is it easy to understand? 
    - Are the variable names descriptive? 
    - Are there any variables created that aren't used? 
    - Is redundant code replaced with function calls? 
    - Is vectorized implementation used instead of nested for loops? 
    - Are classes defined and objects created where applicable? 
    - Are packages used to perform tasks instead of implementing them from scratch?
    
**NOTE:** Your stored models, once loaded, should *just work* when fed with our held-out data (which looks similar to the data we've given you). We won't do any preprocessing before we feed it into the model's `predict(X)` function; `predict(X)` should handle the preprocessing. Pay particular attention to handling the edge cases we've talked about.

Feel free to ask questions to clarify things. Submit everything you tried, not just the things that worked. I encourage you to try and showcase your talents. The more you go above and beyond what's expected, the more impressed we'll be. **Bonus points if you fit Keras/Tensorflow/Pytorch/Caffe models** in addition to your Linear/Tree-based models.

## 0. Import dependencies

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing as scale
from sklearn.utils import resample
from sklearn.model_selection import train_test_split


from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.ensemble import RandomForestClassifier 
from sklearn.naive_bayes import GaussianNB 
from sklearn.metrics import mean_squared_error, accuracy_score, average_precision_score, precision_score, f1_score,recall_score, roc_auc_score

## Task 1
`predictive_maintenance_dataset.csv` is a file that contains parameters and settings (`operational_setting_1`, `operational_setting_2`, `sensor_measurement_1`, `sensor_measurement_2`, etc.) for many wind turbines. There is a column called `unit_number` which specifies which turbine it is, and one called `status`, in which a value of 1 means the turbine broke down that day, and 0 means it didn't. Your task is to create a model that, when fed with operational settings and sensor measurements (`unit_number` and `time_stamp` will *not* be fed in), outputs 1 if the turbine will break down within the next 40 days, and 0 if not.

**NOTE:** The model should output 1 if the turbine is anywhere between 40 and 0 days away from failure, not *only* 40 days from failure.

In [4]:
## What the data that we'll feed into your model's predict(X) function will look like:
# Notice what the operational_setting_3 column looks like
df_X = pd.read_csv("predictive_maintenance_dataset.csv").drop(labels=['status', 'unit_number', 'time_stamp'], axis='columns')
#df_X

### 1. Preprocess data

In [108]:
def preProcessData(data):
    
    df = data
    nullcols = []

    for col in df.columns:
        
        nbnull = (df[col].isnull()*1).sum()
        if nbnull>0: #if there are null values in a column
            t = type(df[df[col].notnull()][col].iat[0]) # type of first non-null value
            #nullcols.append([col,t])
            print(col, nbnull, t)
                
            
            if t == str: #hopefully there's no edge case where a string column isn't categorical!
                df[col].fillna(df[col].mode()[0], inplace=True)
                #if df[col].dtype.name == 'category':
                df = pd.concat([df.drop(col, axis=1), pd.get_dummies(df[col])], axis=1)
                
            #elif t == 'numpy.float64': #numerical
    df.fillna(df.mean(axis=0), axis=0, inplace=True)
                
                #more data types to account off can be added here 
     
    return df

### 2. Setting Labels

We have an interesting case here: where we're checking if a turbine is going to fail in 40 days or less. So essentially we're trying to figure out a problem where given all the parameters what is the likelihood that a certain unit fails within a 40 day timespan. 


So we just have to identify the date the turbines failed and mark the data going back up to a maximum of 40 days as a failure as well.

In [79]:
df.groupby(['status']).count()

,unit_number,time_stamp,operational_setting_1,operational_setting_2,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,sensor_measurement_6,...,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21,High,Low
status,,,,,,,,,,,,,,,,,,,,,
0,143570,143570,143570,143570,143570,143570,143570,143570,143570,143570,...,143570,143570,143570,143570,143570,143570,143570,143570,143570,143570
1,633,633,633,633,633,633,633,633,633,633,...,633,633,633,633,633,633,633,633,633,633


In [6]:
def setLabels(df,y_col, limit=40):
    """
    Function that takes in the dataframe, the target-variable column, 
    and a number defining the window period for failure
    """
    print("Initial split of functioning to non-functioning machines: ", df.groupby(['status']).count())
    df[y_col] = df[y_col].replace(0, np.NaN) #Let's replace all the 0s with NaNs and then we work backwords
    df[y_col] = df[y_col].fillna(method='bfill', limit=40) # fill backward up to 40days. Thankfully the data is frequent and daily
    df[y_col] = df[y_col].fillna('0') #fill the rest with zeros
    print("Final split of functioning to non-functioning machines: ", df.groupby(['status']).count())
    
    return df

## Gotta have this outside the function!

In [83]:
df = df.drop(['time_stamp', 'unit_number'], axis = 1)


In [17]:
def normalize(df):
    columns_to_normalize = df.columns[1:24] 
    standard_sc = scale.StandardScaler()
    x_std = standard_sc.fit_transform(df)
    df_scaled = pd.DataFrame(x_std)
    return df_scaled

### Modelling

In [171]:
def dataPrep(df_scaled, df):
    xtrain, xtest, ytrain, ytest = train_test_split(df_scaled, df['status'], test_size = 0.25, random_state = 19)
    ytrain, ytest = ytrain.astype(int), ytest.astype(int)
    return xtrain, xtest, ytrain, ytest

In [141]:
def metrics(ytest, pred):
    """
    Function to evaluate models against models 
    """
    print('accuracy score: ', accuracy_score(ytest, pred))
    print('RMSE:', mean_squared_error(ytest,pred))
    print('Recall score: ', recall_score(ytest,pred))
    
    print('average_precision_score: ', average_precision_score(ytest,pred))
    print('Precision Score: ',precision_score(ytest,pred))
    print('F1_score: ',f1_score(ytest, pred))
    print('roc_auc_score: ', roc_auc_score(ytest, pred))

In [ ]:
def score(training_model):
    model = training_model.fit(xtrain.values,ytrain.values)
    pred = model.predict(xtest.values)
    metrics(pred,ytest)
    return pred

In [176]:
def logisticRegression(xtrain,xtest,ytrain,ytest):
    LR = LogisticRegression(multi_class='ovr')
    pred = score(LR)
    return pred

def randomForestClassifier(xtrain,xtest,ytrain,ytest,n_estimators=25,min_samples_split=25,max_depth=5,random_state=72):
    RF = RandomForestClassifier(n_estimators = 25, min_samples_split=25, max_depth =5, random_state=72)
    pred = score(RF)
    return pred

def gaussianNaiveBayes(xtrain,xtest,ytrain,ytest):
    GNB = GaussianNB()
    pred = score(GNB)
    return pred

def supportVectorMachine(xtrain,xtest,ytrain,ytest):
    svc = SVC(kernel='linear')
    pred = score(svc)
    return pred

In [ ]:
def train(df_scaled,df):
    
    xtrain, xtest, ytrain, ytest = dataPrep(df_scaled,df)
    
    

In [ ]:
X = pd.read_csv("predictive_maintenance_dataset.csv")

In [178]:
def Predict(X):
    
    df = X.sort_values(by = ['unit_number', 'time_stamp'])
    
    df = preProcessData(df)
    
    df = setLabels(df, 'status', 40)
    
    df = df.drop(['time_stamp', 'unit_number'], axis = 1)
    
    df_scaled = normalize(df)
    
    predictions = 
    
    
    
    

    
    
    

## Task 2
`forecasting_dataset.csv` is a file that contains pollution data for a city. Your task is to create a model that, when fed with columns `co_gt`, `nhmc`, `c6h6`, `s2`, `nox`, `s3`, `no2`, `s4`, `s5`, `t`, `rh`, `ah`, and `level`, predicts the value of `y` six hours later.

**NOTE:** In the data we've given you, the value of `y` for a given row is the value of `y` *for the timestamp of that same row*. We're asking you to predict the value of `y` 6 hours *after the timestamp of that row*.

In [111]:
## What the data that we'll feed into your model's predict(X) function will look like:
# Notice what the level column looks like
pd.read_csv("forecasting_dataset.csv").drop(labels=['date', 'time', 'y'], axis='columns')

,co_gt,nhmc,c6h6,s2,nox,s3,no2,s4,s5,t,rh,ah,level
0,-200.0,-200.0,7.2,867.0,-200.0,834.0,-200.0,1314.0,891.0,14.8,57.3,0.9603,NaN
1,0.5,-200.0,3.9,704.0,-200.0,861.0,-200.0,1603.0,860.0,24.4,65.0,1.9612,Low
2,3.7,-200.0,23.3,1386.0,NaN,626.0,109.0,2138.0,NaN,23.3,38.6,1.0919,High
3,2.1,-200.0,12.1,1052.0,183.0,779.0,NaN,1690.0,952.0,28.5,27.3,1.0479,High
4,4.4,-200.0,21.7,1342.0,786.0,499.0,206.0,1546.0,2006.0,12.9,54.1,0.8003,High
5,-200.0,NaN,7.0,859.0,-200.0,892.0,-200.0,NaN,754.0,17.7,54.0,1.0826,Very low
6,-200.0,-200.0,NaN,1004.0,123.0,818.0,119.0,NaN,783.0,32.4,22.4,1.0701,Very low
7,2.3,-200.0,NaN,1035.0,100.0,746.0,112.0,1611.0,1062.0,27.9,27.6,1.0252,High
8,1.0,-200.0,4.5,737.0,49.0,899.0,63.0,NaN,611.0,25.1,55.7,NaN,Moderate
9,1.9,-200.0,10.2,984.0,110.0,838.0,92.0,1897.0,976.0,26.6,47.6,1.6315,High


In [142]:
df = pd.read_csv("forecasting_dataset.csv").sort_values(by = ['date', 'time'], ascending = True).reset_index()
df

,index,date,time,y,co_gt,nhmc,c6h6,s2,nox,s3,no2,s4,s5,t,rh,ah,level
0,3051,1/1/2005,0:00:00,1046,-200.0,-200.0,4.2,724.0,-200.0,848.0,-200.0,898.0,1201.0,8.2,40.1,0.4375,Very low
1,1909,1/1/2005,10:00:00,973,1.2,-200.0,4.7,748.0,190.0,878.0,97.0,968.0,991.0,4.7,57.2,0.4932,High
2,6211,1/1/2005,11:00:00,1011,1.7,-200.0,5.4,782.0,225.0,846.0,113.0,992.0,1019.0,6.8,48.6,0.4822,High
3,6498,1/1/2005,12:00:00,1097,2.3,-200.0,NaN,875.0,261.0,778.0,125.0,1080.0,1151.0,8.6,43.6,NaN,High
4,5266,1/1/2005,13:00:00,1149,2.2,-200.0,8.4,914.0,382.0,742.0,147.0,NaN,1242.0,9.5,41.2,0.4908,High
5,1524,1/1/2005,14:00:00,1117,2.4,-200.0,8.9,934.0,357.0,721.0,153.0,1075.0,1206.0,10.9,35.9,0.4680,High
6,4195,1/1/2005,15:00:00,1085,2.2,-200.0,7.9,896.0,299.0,760.0,147.0,1049.0,1138.0,12.5,32.3,0.4670,High
7,4670,1/1/2005,16:00:00,1102,2.1,-200.0,7.7,885.0,313.0,772.0,139.0,1051.0,1142.0,12.8,32.6,NaN,High
8,3712,1/1/2005,17:00:00,1281,3.0,-200.0,12.1,1053.0,510.0,659.0,165.0,1192.0,1438.0,10.9,39.7,0.5166,High
9,7296,1/1/2005,18:00:00,1472,4.7,-200.0,16.6,1198.0,832.0,555.0,191.0,1344.0,1735.0,NaN,51.8,0.5961,NaN


The time column doesn't sort properly 1 am shows up after 7. Make sure to change the format! 

In [143]:
df = preProcessData(df)

co_gt 441 <class 'numpy.float64'>
nhmc 434 <class 'numpy.float64'>
c6h6 413 <class 'numpy.float64'>
s2 421 <class 'numpy.float64'>
nox 425 <class 'numpy.float64'>
s3 416 <class 'numpy.float64'>
no2 455 <class 'numpy.float64'>
s4 394 <class 'numpy.float64'>
s5 400 <class 'numpy.float64'>
t 424 <class 'numpy.float64'>
rh 436 <class 'numpy.float64'>
ah 425 <class 'numpy.float64'>
level 418 <class 'str'>


In [144]:
df.isnull().sum()

index        0
date         0
time         0
y            0
co_gt        0
nhmc         0
c6h6         0
s2           0
nox          0
s3           0
no2          0
s4           0
s5           0
t            0
rh           0
ah           0
High         0
Low          0
Moderate     0
Very High    0
Very low     0
dtype: int64

In [117]:
df_scaled = normalize(df.loc[:,'y':])

In [122]:
# might want to fill the NaNs with a moving window average instead

In [145]:
df['time'] = pd.to_datetime(df['time'])

In [146]:
df

,index,date,time,y,co_gt,nhmc,c6h6,s2,nox,s3,...,s4,s5,t,rh,ah,High,Low,Moderate,Very High,Very low
0,3051,1/1/2005,2018-09-28 00:00:00,1046,-200.000000,-200.000000,4.20000,724.00000,-200.00000,848.000000,...,898.000000,1201.000000,8.200000,40.1,0.437500,0,0,0,0,1
1,1909,1/1/2005,2018-09-28 10:00:00,973,1.200000,-200.000000,4.70000,748.00000,190.00000,878.000000,...,968.000000,991.000000,4.700000,57.2,0.493200,1,0,0,0,0
2,6211,1/1/2005,2018-09-28 11:00:00,1011,1.700000,-200.000000,5.40000,782.00000,225.00000,846.000000,...,992.000000,1019.000000,6.800000,48.6,0.482200,1,0,0,0,0
3,6498,1/1/2005,2018-09-28 12:00:00,1097,2.300000,-200.000000,1.13454,875.00000,261.00000,778.000000,...,1080.000000,1151.000000,8.600000,43.6,-7.792764,1,0,0,0,0
4,5266,1/1/2005,2018-09-28 13:00:00,1149,2.200000,-200.000000,8.40000,914.00000,382.00000,742.000000,...,1414.505295,1242.000000,9.500000,41.2,0.490800,1,0,0,0,0
5,1524,1/1/2005,2018-09-28 14:00:00,1117,2.400000,-200.000000,8.90000,934.00000,357.00000,721.000000,...,1075.000000,1206.000000,10.900000,35.9,0.468000,1,0,0,0,0
6,4195,1/1/2005,2018-09-28 15:00:00,1085,2.200000,-200.000000,7.90000,896.00000,299.00000,760.000000,...,1049.000000,1138.000000,12.500000,32.3,0.467000,1,0,0,0,0
7,4670,1/1/2005,2018-09-28 16:00:00,1102,2.100000,-200.000000,7.70000,885.00000,313.00000,772.000000,...,1051.000000,1142.000000,12.800000,32.6,-7.792764,1,0,0,0,0
8,3712,1/1/2005,2018-09-28 17:00:00,1281,3.000000,-200.000000,12.10000,1053.00000,510.00000,659.000000,...,1192.000000,1438.000000,10.900000,39.7,0.516600,1,0,0,0,0
9,7296,1/1/2005,2018-09-28 18:00:00,1472,4.700000,-200.000000,16.60000,1198.00000,832.00000,555.000000,...,1344.000000,1735.000000,9.904877,51.8,0.596100,1,0,0,0,0


Now we shift everything up by 6 hours to generate our labels

In [155]:
df['y'] = df.y.shift(6)

In [157]:
df.drop(df.index[:6], inplace = True)

In [158]:
df

,index,date,time,y,co_gt,nhmc,c6h6,s2,nox,s3,...,s4,s5,t,rh,ah,High,Low,Moderate,Very High,Very low
6,4195,1/1/2005,2018-09-28 15:00:00,None,2.200000,-200.000000,7.90000,896.00000,299.00000,760.000000,...,1049.000000,1138.000000,12.500000,32.3,0.467000,1,0,0,0,0
7,4670,1/1/2005,2018-09-28 16:00:00,None,2.100000,-200.000000,7.70000,885.00000,313.00000,772.000000,...,1051.000000,1142.000000,12.800000,32.6,-7.792764,1,0,0,0,0
8,3712,1/1/2005,2018-09-28 17:00:00,None,3.000000,-200.000000,12.10000,1053.00000,510.00000,659.000000,...,1192.000000,1438.000000,10.900000,39.7,0.516600,1,0,0,0,0
9,7296,1/1/2005,2018-09-28 18:00:00,None,4.700000,-200.000000,16.60000,1198.00000,832.00000,555.000000,...,1344.000000,1735.000000,9.904877,51.8,0.596100,1,0,0,0,0
10,3046,1/1/2005,2018-09-28 19:00:00,None,3.600000,-200.000000,11.40000,1029.00000,622.00000,637.000000,...,1188.000000,1611.000000,8.100000,54.1,0.588200,1,0,0,0,0
11,547,1/1/2005,2018-09-28 01:00:00,None,1.600000,-200.000000,8.80000,930.00000,215.00000,649.000000,...,1024.000000,1617.000000,9.904877,50.7,0.456400,1,0,0,0,0
12,6069,1/1/2005,2018-09-28 20:00:00,None,2.500000,-200.000000,7.90000,897.00000,402.00000,720.000000,...,1072.000000,1436.000000,7.800000,54.6,0.578600,1,0,0,0,0
13,2244,1/1/2005,2018-09-28 21:00:00,None,2.300000,-200.000000,8.10000,898.96525,334.00000,718.000000,...,1104.000000,1389.000000,6.200000,59.6,0.569800,1,0,0,0,0
14,353,1/1/2005,2018-09-28 22:00:00,None,2.100000,-200.000000,6.40000,830.00000,295.00000,765.000000,...,1058.000000,1313.000000,5.700000,59.9,0.552300,1,0,0,0,0
15,6874,1/1/2005,2018-09-28 23:00:00,None,1.700000,-200.000000,1.13454,773.00000,156.62006,820.000000,...,1003.000000,1232.000000,5.600000,59.7,0.546300,1,0,0,0,0
